In [1]:
import numpy as np
import pandas as pd
from collections import Counter
from typing import Union
from datetime import datetime, timedelta 

In [2]:
init_df = pd.read_csv('./data_analytics.csv', parse_dates=['Event Date'])
init_df.tail()

,Event Date,App Apple ID,Subscription Name,Standard Subscription Duration,Promotional Offer Name,Promotional Offer ID,Subscription Offer Type,Subscription Offer Duration,Marketing Opt-In Duration,Customer Currency,Proceeds Currency,Device,Country,Subscriber ID,Subscriber ID Reset,Refund,Purchase Date,Units
2375,2019-08-24,1,weekly 9.99 + 7D trial,7 Days,NaN,NaN,NaN,NaN,NaN,MYR,MYR,iPhone,MY,223411322456766566,NaN,NaN,NaN,1
2376,2019-08-24,1,weekly 9.99 + 7D trial,7 Days,NaN,NaN,NaN,NaN,NaN,USD,USD,iPhone,US,223411322456766569,NaN,NaN,NaN,1
2377,2019-08-24,1,weekly 9.99 + 7D trial,7 Days,NaN,NaN,NaN,NaN,NaN,INR,INR,iPhone,IN,223411322456766571,NaN,NaN,NaN,1
2378,2019-08-24,1,weekly 9.99 + 7D trial,7 Days,NaN,NaN,NaN,NaN,NaN,USD,USD,iPhone,US,223411322456766572,NaN,NaN,NaN,1
2379,2019-08-24,1,weekly 9.99 + 7D trial,7 Days,NaN,NaN,NaN,NaN,NaN,MYR,MYR,iPhone,MY,223411322456766573,NaN,NaN,NaN,1


In [3]:
def get_lifetime_value(init_df: pd.DataFrame, date_period: Union[tuple[datetime, datetime],tuple[datetime, timedelta]],
                       app_id: int, price: float = 9.99):
    
    if isinstance(date_period[1], timedelta):  
        start, interval = date_period
        
        if not isinstance(start, datetime):
            start = datetime.strptime(start, '%Y-%m-%d')
        
        date_period = [start, start + interval]
    
    df = init_df[(init_df['App Apple ID'] == app_id &
             init_df['Event Date'].between(*date_period))]
    
    df = df[['App Apple ID', 'Subscriber ID',
             'Event Date', 'Subscription Offer Type']]
    
    proceeds = len(df[df['Subscription Offer Type'].isna()].index) * price * 0.7
    value_counts = list(df['Subscriber ID'].value_counts().values)  
    
    _, occurances = zip(*sorted(
        Counter(value_counts).items(), key=lambda item: item[0])
    )
    
    total, last_prob = 0, 1
    
    for idx, count in enumerate(occurances):    
        prob = 1 - count / sum(occurances[idx:])
        
        last_prob *= prob
        total = total + last_prob
    
    lifetime_value = total * proceeds
    return lifetime_value

In [5]:
get_lifetime_value(init_df, date_period=['2019-07-15', timedelta(weeks=6)], app_id=1)

12711.757363101075